# PYNQ tutorial: Xlnk and Contiguous Memory Allocation

`Xlnk()` can assign a contiguous block of memory. The data in the memory buffer can be transferred efficiently between the PS and the PL. 

Note: The `Xlnk()` driver is overlay-independent, meaning it can be used no matter what overlay you are using.

## Preparation

In the following cell we will define a couple of useful functions.

In [1]:
import re
from time import sleep
from pprint import pprint

def get_kb(mmu):
    return int(mmu.cma_stats()['CMA Memory Available']//1024)

def get_bufcount(mmu):
    return int(memmanager.cma_stats()['Buffer Count'])

def print_kb(mmu):
    print("Available Memory (KB): " + str(get_kb(mmu)))

def print_virtual_address(virtual_ptr):
    str1 = repr(virtual_ptr)
    print(re.findall(r'0x[0-9A-F]+', str1, re.I)[0])

def print_physical_address(physical_ptr):
    print("0x" + format(physical_ptr, '02x'))

## Step 1: Create an instance of Xlnk
Xlnk instance will be labeled `mmu` (Memory Management Unit) throughout this notebook.

In [2]:
from pynq import Xlnk
mmu = Xlnk()

## Step 2: Check the status

In [3]:
pprint(mmu.cma_stats())

{'Buffer Count': 0, 'CMA Memory Available': 37330944, 'CMA Memory Usage': 0}


## Step 3: Allocate the memory

In [4]:
print("Before memory allocation:")
print_kb(mmu)

virtual_ptr = mmu.cma_alloc(1024*1024)

print("After memory allocation:")
print_kb(mmu)

Before memory allocation:
Available Memory (KB): 36456
After memory allocation:
Available Memory (KB): 35464


## Step 4: Check the memory buffer address

The virtual address can be used by Linux. The Physical address can be passed to a peripheral in the PL.

In [5]:
print("Buffer pointer address (virtual memory):")
print_virtual_address(virtual_ptr)

physical_ptr = mmu.cma_get_phy_addr(virtual_ptr)

print("Buffer pointer address (physical memory):")
print_physical_address(physical_ptr)

Buffer pointer address (virtual memory):
0x2f768000
Buffer pointer address (physical memory):
0x16900000


## Step 5: Free the memory
It is always a good practice to free the contiguous memory after use. This prevents memory leaks from the program.

In [6]:
mmu.cma_free(virtual_ptr)
sleep(1)
print(mmu.cma_stats())

{'CMA Memory Available': 36773888, 'CMA Memory Usage': 0, 'Buffer Count': 0}


In [7]:
print_kb(mmu)

Available Memory (KB): 35912


Note that there might be a difference between the currently available memory and the available memory before allocation. This is normal. The OS will eventually free up all the available memory.